# Visualizations

In [ ]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
from html2text import html2text
from wordcloud import WordCloud, STOPWORDS


## Pandas Dataframes

We can use [django pandas](https://github.com/chrisdev/django-pandas) to create data frames from django models.

By overriding the default manager of a model with a `django_pandas.managers.DataFrameManager` you can use its `to_dataframe` method for creating data frames.

In [ ]:
df = MeetupEvent.objects.filter(group__urlname="Django-Vienna").annotate(
    visitors=Count("attendants")
).order_by("event_date").to_dataframe(
    ["name", "venue", "visitors"], index="event_date"
)
df

With matplotlib installed it is very easy to generate charts from dataframes by calling there `plot` method

In [ ]:
sns.set_style("whitegrid")
p = df.plot(kind="bar", figsize=(16,10), title="Visitors at Django-Vienna meetups")



## Creating word clouds

Lets create word clouds with [WordCloud](https://amueller.github.io/word_cloud/)

### A word cloud of most regular meetup participants

In [ ]:
MeetupMember.objects.annotate(visits=Count("events")).values_list(
    "name", "visits"
).order_by("-visits").to_dataframe(["visits"], index="name").head(n=10)


In [ ]:
# create a dictionary with member names and their visits
frequencies = dict(MeetupMember.objects.annotate(visits=Count("events")).values_list("name", "visits"))

visitorcloud = WordCloud(width=700, height=700).fit_words(frequencies)

plt.figure(figsize=(16, 16))
plt.imshow(visitorcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### A word cloud of all words used in the events descriptions

In [ ]:
grp = MeetupGroup.objects.get(urlname="Django-Vienna")
text = html2text(grp.description)

# create a big text containing the meetup groups description + the description of all their events.
for event in grp.events.all():
    if event.description:
        text += " " + html2text(event.description)

# remove newlines and some characters
replace_patter = r"[\s~*!\?\\\[\]\(\)<>]+"
text = re.sub(replace_patter, " ", text)
text

In [ ]:
stopwords = {"https", "http"}

descriptions = WordCloud(
    width=1000, 
    height=500, 
    relative_scaling=.5, 
    stopwords=STOPWORDS | stopwords
).generate(text)

plt.figure(figsize=(16,8))
plt.imshow(descriptions, interpolation='bilinear')
plt.axis("off")
plt.show()